In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from glob import glob
import os

### Identify input data files using Glob library

In [2]:
xl_files = glob('./Exam1-fa19/datafiles-socrative/*.xlsx', recursive=False)
display(xl_files)

['./Exam1-fa19/datafiles-socrative/Class_09_12_2019__15_02_QZ_datascience20190912thu.xlsx',
 './Exam1-fa19/datafiles-socrative/Class_08_29_2019__14_52_QZ_datascience20190829thu.xlsx',
 './Exam1-fa19/datafiles-socrative/Class_09_17_2019__15_01_QZ_datascience20190917tuesday.xlsx',
 './Exam1-fa19/datafiles-socrative/Class_09_19_2019__15_01_QZ_datascience20190919thursday.xlsx',
 './Exam1-fa19/datafiles-socrative/Class_09_10_2019__15_00_QZ_datascience20190910.xlsx']

#### Define necessary functions

Since the code will run in a loop, much of the work will be pushed into functions which are called on each loop.

In [18]:
# define a list of default necessary columns 
necessary_cols = ["Student Names", "Student ID"]

def identify_header_row(df, req_cols = necessary_cols):
    """
    given a pandas dataframe, returns the row number which is presumably the header. 
    
    req_cols: optional, list of strings used to identify the header row
    """
    # make a condition which: 
    #     - selects all columns containing string
    #     - stacks those columns into rows and checks any which are in req_cols
    #     - returns the df to inital shape, as a boolean mask named condition
    condition = df.select_dtypes(include=[object]).stack().isin(req_cols).unstack()
    # fill any empty values in the conditional bool mask with "False"
    condition = condition.fillna(False)
    # select from df those data meeting boolean mask conditions
    # and drop any rows which have ALL False, or NaN (row wise)
    header_row = df[condition].dropna(how='all')
    # Verify the quantity of rows is valid
    header_row_qty = len(header_row)
    if header_row_qty > 1:
        raise Exception(f'Too many header rows!')
    elif header_row_qty < 1:
        raise Exception('No header rows found!')
    # return the index position of the row as an int
    return header_row.index.values[0] + 1

def retrieve_longest_str(group):
    """
    given a groupby series returns the longest string of the series
    transformed to title case.
    """
    # retrieve the largest result using the len function as the key to determine largest
    try:
        longest = max(group, key=len)
        longest = longest.title()
    except TypeError:
        longest = group[0]
    return longest

def collapse_date_floats(group):
    """
    given a groupby series of floats, returns 1 if any are 1 and
    returns 0 if none are 1
    """
    
    if any(group) > 0:
        return 1
    else:
        return 0

def retrieve_common_name(group):
    """
    give a groupby series of strings, returns the most frequent
    """
    
    return max(set(group), key=group.count)

#### load each file, run initial cleaning functions then concatenate them into a single (pre-cleaned) df.

In [19]:
df = pd.DataFrame()

for filePath in xl_files:
    path, fileName = os.path.split(filePath)
       
    # extract information based on expected file naming convention
    class_date = fileName.split('__')[0]  # split on all instances of __, and keep the first value   
    date = class_date.split('_', 1)[-1]  # split on only the first instance of "_" and keep the final value
    date = date.replace('_', '-')  # make date look more ... nice

    # load in the file as a pandas dataframe
    current_date_df = pd.read_excel(filePath)
        
    # retrieve the header row for current file
    header_index = identify_header_row(current_date_df)
    
    # load in the file again, now stating the proper header row
    current_date_df = pd.read_excel(filePath, header=header_index, na_values='0')

    # the dataframes load in with a score row.. we don't need them
    current_date_df = current_date_df.loc[current_date_df['Student Names'] != 'Class Scoring']

    # rename netID col
    current_date_df['netID'] = current_date_df['My UTC netID is']

    # keep only those cols which are necessary, this also allows us to easily re-order the cols
    keep_cols = ['netID', 'Student Names']
    current_date_df = current_date_df[keep_cols]

    # add in the date we derived from the filename
    current_date_df[date] = 1

    # drop any cols which are fully nan
    current_date_df = current_date_df.dropna(axis=1, how='all')

    # set utcID col to upper case, this helps align inconsistant casing
    current_date_df['netID'] = current_date_df['netID'].str.upper()

    # append the current_data_df to the df created to hold results
    df = df.append(current_date_df, ignore_index=True, sort = False)

#### Now that the df is merged, aggregate details based on pandas 'groupby' function


In [20]:
# identify if any student names which have no netID saved
problemIDs = df['Student Names'].loc[df['netID'].isna()]

# if so, solve it with below block
if len(problemIDs) > 0:
    # fill in nan netIDs with most common (mode) netID from student name groupby object
    df['netID'] = df.groupby(['Student Names'])['netID'].apply(lambda x: x.fillna(x.mode()[0]))

# verify no remaining problematic IDs exist. If it does, raise an exception
problemIDs = df['Student Names'].loc[df['netID'].isna()]
if len(problemIDs) > 0:
    raise Exception('ProblemIDs still exist!')

# fill nans with 0.0
df.fillna(0.0, inplace=True)

# groupby the netID, and assign the longest string from the group back to Student Names
# transforming the groupby objects returns the proper series which can be assigned to the column
# this aligns the Student name variants
df['Student Names'] = df['Student Names'].groupby(df['netID']).transform(retrieve_longest_str)

# filtered list comprehension which returns all df columns excepting student names & netID
date_cols = [x for x in df.columns if x not in ['Student Names', 'netID']]

# iterate over dates collapsing duplicate names
for date in date_cols:
    df[date] = df[date].groupby(df['netID']).transform(collapse_date_floats)

df.rename({'Student Names': 'name'}, axis=1, inplace=True)    

# now drop any duplicate rows
df.drop_duplicates(inplace=True)

In [21]:
display(df)

,netID,name,09-12-2019,08-29-2019,09-17-2019,09-19-2019,09-10-2019
0,YMP337,Andrew Turgeson,1.0,1.0,1.0,1.0,1.0
1,FZP281,Godfred Sabbih,1.0,1.0,1.0,1.0,1.0
2,NKN328,Noah Gaston,1.0,1.0,1.0,0.0,1.0
3,CCN399,Paul Smith,1.0,1.0,1.0,1.0,1.0
4,LHF446,Samuel Tucker Clark,1.0,1.0,1.0,1.0,1.0
6,SXX597,"Yost, Mason",1.0,1.0,1.0,1.0,1.0
7,QVD441,"Caleb, Powell",1.0,1.0,1.0,1.0,0.0
14,CRR434,"Vandergriff, Taylor",0.0,1.0,1.0,0.0,1.0


#### Save the results

In [13]:
df.to_excel('midterm-report.xlsx', index=False)